In [1]:
import datetime as dt
from funcy import walk_values, join, flatten, first, rest, rpartial

import pandas as pd
import numpy as np

try:
    import plotly.plotly as py
    import plotly.graph_objs as go
    import cufflinks as cf
except:
    !pip install plotly
    !pip install cufflinks

In [2]:
# helpers
from toolz import keyfilter

def keep(d, whitelist):
    return keyfilter(lambda k: k in whitelist, d)

def omit(d, blacklist):
    return keyfilter(lambda k: k not in blacklist, d)

In [3]:
import requests

def historic_data(urls=None):
    return list(map(lambda x: requests.get(x).json(), urls))

def merge_historic_data(historic_data):
    """ Simplify and flatten all historic data into a single list of events."""
    data = []
    for hist in historic_data:
        data.append([simplify_fragment(x) for x in hist['history'].values()])
    return list(flatten(data))

def simplify_fragment(obj):
    """ Simplify and flatten individual fragment."""
    # clean up the mess
    def replace_values(val):
        if type(val) == dict:
            return walk_values(replace_values, val)
        if val == "?" or val == 'None':
            return 0
        return val
    obj = walk_values(replace_values, obj)
    
    return {
        'symbol': obj['symbol'],
        'category': obj['category'],
        'supply': obj['availableSupply'],
        'position': int(obj['position']),
        'cap_usd': round(float(obj['marketCap']['usd'])),
        'cap_btc': round(float(obj['marketCap']['btc'])),
        'volume_btc': round(float(obj['volume24']['btc'])),
        'price_usd': float(obj['price']['usd']),
        'price_btc': float(obj['price']['btc']),
        'timestamp': dt.datetime.fromtimestamp(obj['timestamp'])

    }


chart_filter = rpartial(
    keep,
    ['symbol', 'timestamp', 'price_usd', 'price_btc', 'cap_usd', 'volume_btc', 'supply'],
)

def simplify_hist_data(historic_data):
    return [keep(x, ['symbol', 'timestamp', 'price_usd']) for x in historic_data]

In [4]:
steem_urls = [
    'http://coinmarketcap.northpole.ro/api/v6/history/STEEM_2016.json',
    'http://coinmarketcap.northpole.ro/api/v6/history/STEEM_2017.json',
]
from toolz import thread_last

steem_chart = thread_last(
    historic_data(steem_urls),
    merge_historic_data,
    (map, chart_filter),
    list,
)

In [5]:
df = pd.DataFrame(steem_chart)

In [6]:
df.sort_values('timestamp').tail()

,cap_usd,price_btc,price_usd,supply,symbol,timestamp,volume_btc
412,483796935,0.000764,2.06669,234092648,STEEM,2017-06-08 00:00:02,14070
413,525291685,0.000808,2.24552,233928749,STEEM,2017-06-09 00:00:03,4593
414,589008115,0.000895,2.51773,233944114,STEEM,2017-06-10 00:00:03,6633
415,555013292,0.000833,2.37201,233984381,STEEM,2017-06-11 00:00:02,4888
416,530551374,0.000768,2.26676,234057145,STEEM,2017-06-12 00:00:02,2742


In [7]:
# cf.go_offline()

In [8]:
df.sort_values('timestamp', inplace=True)
df.set_index('timestamp', inplace=True)

In [9]:
df[['cap_usd', 'supply']].iplot(title='USD Market Cap and STEEM Supply',
                 colors=['blue', 'orange'],
                 theme='white',
                 fill=True,
                 filename='steem-supply-cap')

/Users/user/anaconda/lib/python3.6/site-packages/cufflinks/plotlytools.py:156: FutureWarning:

pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp

